In [ ]:
!pip install transformers evaluate sentencepiece accelerate scikit-multilearn peft datasets sentence-transformers bitsandbytes trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavio

In [ ]:
import datasets
from datasets import load_dataset

dataset = load_dataset('knowledgator/events_classification_biotech')

README.md:   0%|          | 0.00/4.99k [00:00<?, ?B/s]

events_classification_biotech.py:   0%|          | 0.00/4.71k [00:00<?, ?B/s]

KeyboardInterrupt: Interrupted by user

In [ ]:
dataset

In [ ]:
classes = [class_ for class_ in dataset['train'].features['label 1'].names if class_]
class2id = {class_:id for id, class_ in enumerate(classes)}
id2class = {id:class_ for class_, id in class2id.items()}

NameError: name 'dataset' is not defined

In [ ]:
import os
import random
import functools
import csv
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from sklearn.metrics import f1_score
from skmultilearn.model_selection import iterative_train_test_split
from datasets import Dataset, DatasetDict, load_dataset
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model
)
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer
)


In [ ]:
ds = load_dataset('bhujith10/multi_class_classification_dataset')
ds

README.md:   0%|          | 0.00/510 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/10.8M [00:00<?, ?B/s]

val-00000-of-00001.parquet:   0%|          | 0.00/692k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16771 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/1056 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3145 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 16771
    })
    val: Dataset({
        features: ['text', 'labels'],
        num_rows: 1056
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 3145
    })
})

We use a multi-label classification dataset from Kaggle https://www.kaggle.com/datasets/shivanandmn/multilabel-classification-dataset/data where the objective is to classify research papers into 6 different subjects or labels. We are provided with the title and abstract and every research paper can belong to more than 1 subject simultaneously. There is a minor difference between multi-label and multi-class problems. In the latter, each sample belongs to only one class from a set of multiple classes. There are many resources available on using transformer based models for binary sequence classification tasks. But, very few resources are available for multi-label problems. In this article, I will explain some of the techniques of how transformer based models can be used in multi-label classification tasks. Specifically, this article deals with 3 different techniques that uses encoder only models. In an another article, I will explain how we can use decoder only models like GPT, Llama, etc., for the same objective. Warning: No new techniques are demonstrated in this article. Many of those reading this article might be aware about the techniques. But, while I was exploring about this problem, there were many resources in an unorganized manner. This article organizes the different techniques in a single place.  

In [ ]:
def load_csv():
  df = pd.read_csv('/content/drive/MyDrive/multilabel_classification_dataset/train.csv')
  return df

def create_huggingface_dataset(push_to_hub=False,dataset_name='multi_class_classification_dataset'):
  df = load_csv()
  data = df.iloc[:,:].values
  idx, text, labels = list(zip(*[(int(row[0]), f"Title: {row[1].strip()},\nAbstract: {row[2].strip()}", row[3:]) for row in data]))
  labels = np.array(labels, dtype=int)
  text = list(text)
  row_ids = np.arange(len(labels))
  row_ids = row_ids.reshape(-1,1)
  train_idx,y_train,val_idx,y_val = iterative_train_test_split(row_ids, labels, test_size=0.2)
  x_train = [text[idx] for idx in train_idx.flatten()]
  x_val = [text[idx] for idx in val_idx.flatten()]

  val_idx_final, y_val_final, test_idx, y_test = iterative_train_test_split(val_idx, y_val, test_size=0.75)

  # Create the final train and test data from the indices
  x_val_final = [text[idx] for idx in val_idx_final.flatten()]
  x_test = [text[idx] for idx in test_idx.flatten()]

  ds = DatasetDict({
      'train': Dataset.from_dict({'text': x_train, 'labels': y_train}),
      'val': Dataset.from_dict({'text': x_val_final, 'labels': y_val_final}),
      'test': Dataset.from_dict({'text': x_test, 'labels': y_test})
      })

  if push_to_hub:
    ds.push_to_hub(f"bhujith10/{dataset_name}")

  return ds

In [ ]:
df = load_csv()

Below is a snap of how the data looks. There is class imbalance in the dataset. We are going to finetune the transformers using Huggingface transformers library and hence it requires the dataset in a specific format. We will perform some processing such as concatenating title and abstract followed by converting the labels into one hot encoded format. We split the dataset into training, testing and validation sets and convert it into Huggingface datasets format.

In [ ]:
df.head()

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0


In [ ]:
df[['Computer Science', 'Physics', 'Mathematics',
       'Statistics', 'Quantitative Biology', 'Quantitative Finance']].sum().reset_index().rename(columns={'index':'field',0:'count'})

,field,count
0,Computer Science,8594
1,Physics,6013
2,Mathematics,5618
3,Statistics,5206
4,Quantitative Biology,587
5,Quantitative Finance,249


In [ ]:
ds = create_huggingface_dataset()

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 16771
    })
    val: Dataset({
        features: ['text', 'labels'],
        num_rows: 1055
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 3146
    })
})

In [ ]:
ds['train'][0]

{'text': 'Title: Rotation Invariance Neural Network,\nAbstract: Rotation invariance and translation invariance have great values in image\nrecognition tasks. In this paper, we bring a new architecture in convolutional\nneural network (CNN) named cyclic convolutional layer to achieve rotation\ninvariance in 2-D symbol recognition. We can also get the position and\norientation of the 2-D symbol by the network to achieve detection purpose for\nmultiple non-overlap target. Last but not least, this architecture can achieve\none-shot learning in some cases using those invariance.',
 'labels': [1, 0, 0, 0, 0, 0]}

In [ ]:
import os
import time
import torch
import pandas as pd
import numpy as np
from datetime import datetime
from collections import defaultdict
from dataclasses import dataclass
from torch.utils.data import DataLoader
from scipy.special import expit as sigmoid
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from skmultilearn.model_selection import iterative_train_test_split
from trl import SFTTrainer
from datasets import Dataset, load_dataset, DatasetDict
from peft import LoraConfig, get_peft_model, TaskType, PeftModel
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, DataCollatorForLanguageModeling

In [ ]:
@dataclass
class Config:
  checkpoint:str = "microsoft/deberta-v3-base"
  max_length:int = 512
  num_labels:int = 6
  problem_type:str = "multi_label_classification"
  lora_rank:int = 8
  lora_alpha:int = 32
  lora_dropout:float = 0.1
  lora_bias:str = "none"
  device:str = "cuda" if torch.cuda.is_available() else "cpu"
  repo_user_id:str = "bhujith10"
  model_name:str = ""
  local_save_path:str = ""
  bf16:bool = torch.cuda.is_available() and torch.cuda.is_bf16_supported()
  fp16:bool = torch.cuda.is_available() and not torch.cuda.is_bf16_supported()
  load_in_4bit:bool = True,
  bnb_4bit_quant_type:bool = "nf4",
  bnb_4bit_compute_dtype:bool = "float16",
  bnb_4bit_use_double_quant:bool = False,
  num_train_epochs:int = 2,
  batch_size:int = 8,
  gradient_accumulation_steps:int = 2,
  gradient_checkpointing:bool = True

config = Config()

In [ ]:
complete_results = defaultdict(dict)

### Load dataset from huggingface

In [ ]:
ds = load_dataset('bhujith10/multi_class_classification_dataset')

README.md:   0%|          | 0.00/510 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/10.8M [00:00<?, ?B/s]

val-00000-of-00001.parquet:   0%|          | 0.00/692k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16771 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/1056 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3145 [00:00<?, ? examples/s]

In [ ]:
# ds['train'] = ds['train'].select(range(100))
# ds['val'] = ds['val'].select(range(100))
# ds['test'] = ds['test'].select(range(100))

### Function to tokenize the dataset

In [ ]:
def tokenize_dataset(tokenizer, config, dataset_name='bhujith10/multi_class_classification_dataset'):

  def tokenize(batch):
    return tokenizer(batch['text'], truncation=True, padding=True, max_length=config.max_length, return_tensors='pt')

  ds = load_dataset(dataset_name)

  ds_encoded = ds.map(tokenize, batched=True, batch_size=None)

  for split in ds_encoded:
    ds_encoded[split].set_format('torch')

  # Convert the labels into float datatype
  ds_encoded = ds_encoded.map(lambda x: {"labels_f": x["labels"].to(torch.float)},remove_columns=["labels"])
  ds_encoded = ds_encoded.rename_column("labels_f", "labels")

  return ds_encoded

### Function to get quantization config

In [ ]:
def quantization_config(config):
  bnb_config = BitsAndBytesConfig(
      load_in_4bit=config.load_in_4bit,
      bnb_4bit_quant_type=config.bnb_4bit_quant_type,
      bnb_4bit_compute_dtype=config.bnb_4bit_compute_dtype,
      bnb_4bit_use_double_quant=config.bnb_4bit_use_double_quant,
  )
  return bnb_config

### Function to load model and tokenizer

In [ ]:
def load_model_and_tokenizer(config,
                             add_pad_token=False,
                             quantization=False,
                             peft=False,
                             load_model_for_sequence_classification=False):

  tokenizer = AutoTokenizer.from_pretrained(config.checkpoint)

  if add_pad_token:
    tokenizer.add_special_tokens({"pad_token":"<pad>"})

  tokenizer.padding_side = "right"

  if load_model_for_sequence_classification:

    model =  AutoModelForSequenceClassification.from_pretrained(
      pretrained_model_name_or_path=config.checkpoint,
      quantization_config=quantization_config(config) if quantization else None,
      torch_dtype=torch.bfloat16 if config.bf16 else torch.float16,
      num_labels=config.num_labels,
      problem_type=config.problem_type
    )

  else:
    model = AutoModel.from_pretrained(config.checkpoint)

  if add_pad_token:
    model.resize_token_embeddings(len(tokenizer))
    model.config.pad_token_id = tokenizer.pad_token_id

  if peft:
    peft_config = LoraConfig(
        task_type=TaskType.SEQ_CLS, r=config.lora_rank, lora_alpha=config.lora_alpha, lora_dropout=config.lora_dropout, bias=config.lora_bias,
      )

    model = get_peft_model(model, peft_config)

  return model, tokenizer

### Function to extract hidden states from model

In [ ]:
def extract_hidden_states(batch,**kwargs):
  model = kwargs.get("model")
  tokenizer = kwargs.get("tokenizer")
  model.to("cuda")

  inputs = {k:v.to("cuda") for k,v in batch.items() if k in tokenizer.model_input_names}

  with torch.no_grad():
    last_hidden_state = model(**inputs).last_hidden_state

  return {"hidden_state": last_hidden_state[:,0].cpu().numpy()}

### Function to calculate micro and macro F1 scores

In [ ]:
def calculate_f1_score(y_true,y_pred):
  clf_dict = classification_report(y_true, y_pred, zero_division=0, output_dict=True)

  return {"micro f1": clf_dict["micro avg"]["f1-score"],
          "macro f1": clf_dict["macro avg"]["f1-score"]}

### Function to encode subjects

In [ ]:
def return_encodings(sample, predictions_column):
  subjects = ['Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance']
  predicted_labels = [0,0,0,0,0,0]

  for i,subject in enumerate(subjects):
    if subject in sample[predictions_column]:
      predicted_labels[i] = 1
    else:
      predicted_labels[i] = 0

  sample['predicted_labels'] = predicted_labels
  return sample

### Function to build classifier using hidden states

In [ ]:
def build_classifier_using_hidden_states(ds, model, tokenizer):
  train_ds = ds['train'].map(extract_hidden_states, batched=True, batch_size=4, fn_kwargs={"model": model,
                                                                                         "tokenizer": tokenizer
                                                                                           })

  x_train = np.array(train_ds["hidden_state"])
  y_train = np.array(train_ds["labels"])

  random_forest_clf = RandomForestClassifier(n_estimators=500)
  multi_class_clf = OneVsRestClassifier(random_forest_clf)
  multi_class_clf.fit(x_train, y_train)

  start_time = time.time()

  test_ds = ds['test'].map(extract_hidden_states, batched=True, batch_size=4, fn_kwargs={"model": model,
                                                                                         "tokenizer": tokenizer})
  x_test = np.array(test_ds["hidden_state"])
  y_test = np.array(test_ds["labels"])
  y_pred = multi_class_clf.predict(x_test)
  f1_score_results = calculate_f1_score(y_test, y_pred)

  end_time = time.time()

  time_for_test_set_prediction = end_time - start_time

  return f1_score_results, time_for_test_set_prediction


### Function to generate predictions using the BERT model


In [ ]:
def generate_predictions_using_bert_models(model,ds):
  dataloader = DataLoader(ds, batch_size=8)

  model.to(config.device)

  all_outputs = []
  actual_outputs = []

  start_time = time.time()
  for batch in dataloader:
      # Move each batch to the same device as the model
      batch = {key: batch[key].to(config.device) for key in ['input_ids','attention_mask','labels']}

      with torch.no_grad():
          outputs = model(**batch)

      output = (sigmoid(outputs.logits.detach().type(torch.float).cpu().numpy())>0.5).astype(float)

      all_outputs.append(output)
      actual_outputs.append(batch['labels'].detach().cpu().numpy())

  end_time = time.time()
  time_taken = end_time - start_time

  all_outputs = np.concatenate(all_outputs, axis=0)
  actual_outputs = np.concatenate(actual_outputs, axis=0)

  f1_score_results = calculate_f1_score(actual_outputs, all_outputs)

  return f1_score_results, time_taken

### Function to compute metrics at the time of finetuning

This is multilabel classification problem i.e. an input can belong to multiple classes.

Hence the sigmoid function converts the logits into probabilities, but does not ensure that the probabilities sum up to 1.

Eg: [1.23, -0.2] -> [0.7, 0.01] - Here the outputs need not add up to 1

In [ ]:
def compute_metrics(pred):
  y_true = pred.label_ids
  y_pred = sigmoid(pred.predictions)
  y_pred = (y_pred > 0.5).astype(float)

  clf_dict = classification_report(y_true, y_pred, zero_division=0, output_dict=True)

  return {"micro f1": clf_dict["micro avg"]["f1-score"],
          "macro f1": clf_dict["macro avg"]["f1-score"]}

### Function to finetune the model

In [ ]:
def finetune_model(model,tokenizer,ds,config):

  num_train_epochs = config.num_train_epochs
  train_batch_size = config.batch_size
  gradient_accumulation_steps = config.gradient_accumulation_steps
  max_steps = int((len(ds['train'])*num_train_epochs)/(train_batch_size*gradient_accumulation_steps))

  training_args = TrainingArguments(
      output_dir=f"./{config.model_name}_results",
      max_steps=max_steps,
      learning_rate=3e-5,
      lr_scheduler_type="cosine",
      per_device_train_batch_size=train_batch_size,
      per_device_eval_batch_size=train_batch_size,
      gradient_accumulation_steps=gradient_accumulation_steps,
      gradient_checkpointing=config.gradient_checkpointing,
      weight_decay=0.01,
      warmup_ratio=0.1,
      evaluation_strategy="steps",
      save_strategy="epoch",
      logging_strategy="steps",
      logging_steps=100,
      save_steps=100,
      eval_steps=100,
      report_to="wandb",
      run_name=f"{config.repo_user_id}/{config.model_name}_results",
      fp16=config.fp16,
      bf16=config.bf16
  )

  trainer = Trainer(
      model=model,
      tokenizer=tokenizer,
      args=training_args,
      train_dataset=ds["train"],
      eval_dataset=ds["val"],
      compute_metrics=compute_metrics
  )

  trainer.train()

  trainer.model.save_pretrained(config.local_save_path)

  return trainer.model

### Function to calculate model size

In [ ]:
def calculate_model_size(model):
  total_params = sum(p.numel() for p in model.parameters())
  trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
  param_size_in_bytes = sum(p.numel() * p.element_size() for p in model.parameters())
  param_size_in_mb = param_size_in_bytes / (1024 ** 2)

  return total_params, trainable_params, param_size_in_mb

### Function to update model related info in config

In [ ]:
def update_model_related_settings(checkpoint, config):
  config.checkpoint = checkpoint
  model_name = checkpoint.split('/')[-1]
  currtime = datetime.now().strftime("%Y_%m_%d_%H_%M")
  config.model_name = f"{model_name}_{currtime}"
  config.local_save_path = config.model_name
  return config

### Function to upload model to huggingface

In [ ]:
def upload_model_to_huggingface(finetuned_model, config, tokenizer=None, peft=False, quantized=False):
  if peft and not quantized:
    finetuned_model.push_to_hub(f"{config.repo_user_id}/{config.model_name}_adapter", safe_serialization=True, max_shard_size='1GB')
    model = finetuned_model.merge_and_unload()

  elif peft and quantized:
    base_model, tokenizer = load_model_and_tokenizer(config,
                                                     quantization=False,
                                                     add_pad_token=False,
                                                     peft=False,
                                                     load_model_for_sequence_classification=True)
    model = PeftModel.from_pretrained(base_model, finetuned_model)
    model = model.merge_and_unload()

  else:
    model = finetuned_model

  model.push_to_hub(f"{config.repo_user_id}/{config.model_name}", safe_serialization=True, max_shard_size='1GB')
  tokenizer.push_to_hub(f"{config.repo_user_id}/{config.model_name}")

In 2018, Google introduced BERT - Bidirectional Encoder Representation from Transformers. How it is different from the other transformer models? The original transformers paper introduced models with both encoder and decoder components. Currently, the models on the limelight such as GPT and LLAMA models have only the decoder components and their primary focus is on generating text. BERT style models have only the encoder component and a key distinction is that while performing attention - the heart of transformers, these models can understand or attend to information on both left and right i.e. in decoder models, while performing attention, a word can't attend to the word on the right. That is how it has been trained. But BERT can attend to words on both left and right. This gives the models the ability to better understand text and generate better embedding representations. Hence these models are good at understanding text rather than generating text. These BERT style models are currently being used for generating embeddings for sentences. How can we use these BERT based models in our case?

There are different ways in which the BERT model was pretrained. Two key techniques includes:

Masked Language Modeling (MLM) - A small fraction of words are randomly masked in the input sentences and the model learns to predict these masked tokens. In order to achieve this BERT models gives attention to both left and right tokens which provides it the ability to generate better embedding representations as mentioned above. We tokenize the sentences before feeding to the model. The masked words will be replaces with special tokens called MASK tokens and the model will be trained to predict these particular tokens.

Next Sentence Prediction (NSP) - Given a pair of sentences, BERT model tries to predict whether the second sentence logically follows the first sentence or not.  Similar to the previous case here, a special token named CLS token is added to the sentence before feeding to the model. As the models gets trained, this token which is the initial token starts to capture the meaning of the whole sequence.

This CLS token which captures the essence of a sentence will serve as inputs to our classifier model. This method is very simple.

Tokenize the sentences and run through the BERT model.

Extract the embeddings corresponding to the CLS token.

Build a classifier that takes these embeddings as input and returns a prediction.

This is not the only way to extract embedding for a sentence. We can perform mean or max pooling of the individual tokens in the sentences to arrive at the embedding of the sentence as whole. There is a package called Sentence Transformer to help us with this. Also, Huggingface library uses CLS token as default for classification tasks.

Sorry for not providing a very detailed info on how BERT models work and how they were trained. There are numerous articles which explain them in an excellent manner.

Resources

https://blog.paperspace.com/bert-pre-training-of-deep-bidirectional-transformers-for-language-understanding/



Below, you can see how tokenizer adds special tokens while tokenizing.

In [ ]:
tokenizer.tokenize("The customer service was great",add_special_tokens=True)

We evaluate the different models on F1 scores and the training dataset has around 16000 samples and testing dataset has around 3500 samples.

We load the BERT model and its tokenizer from Huggingface and tokenize the dataset. The tokenized dataset is run one by one through the BERT model and we extract the vectors corresponding to the CLS token. CLS token is always the first token when a sentence is tokenized. This vector will serve as input to the classifier. Here, I have used Random Forest Classifier with OneVsRest classifier that will act as a wrapper which will help us in converting the multi-label classification problem into a series of binary classification problems. For example, in our case we have 6 different labels and hence underneath the OneVsRest classifier we will have 6 different Random Forest models predicting whether the input belongs to a particular label or not. This is just an example illustrating this technique. You can try out different models to improve performance.

In [ ]:
###### FOR ARTICLE ######
## Add this below code
## Add the functions extract_hidden_states() and build_classifier_using_hidden_states()

# checkpoint = 'microsoft/deberta-v3-base'

# tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# model = AutoModel.from_pretrained(checkpoint)



## Build classifier that uses hidden states from bert model

In [ ]:
checkpoint = 'google-bert/bert-large-uncased'
config = update_model_related_settings(checkpoint, config)

model, tokenizer = load_model_and_tokenizer(config=config,
                                            add_pad_token=False,
                                            peft=False,
                                            load_model_for_sequence_classification=False
                                            )

ds = tokenize_dataset(tokenizer,
                      config)

f1_score_results, time_taken = build_classifier_using_hidden_states(ds, model, tokenizer)

complete_results['hidden_states_classifier'][config.model_name] = {**f1_score_results, 'time_taken': time_taken}

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

...

Map:   0%|          | 0/16771 [00:00<?, ? examples/s]

Map:   0%|          | 0/1056 [00:00<?, ? examples/s]

Map:   0%|          | 0/3145 [00:00<?, ? examples/s]

Map:   0%|          | 0/16771 [00:00<?, ? examples/s]

Map:   0%|          | 0/3145 [00:00<?, ? examples/s]

In [ ]:
complete_results

defaultdict(dict,
            {'hidden_states_classifier': {'bert-large-uncased_2024_12_04_07_10': {'micro f1': 0.7470395206163504,
               'macro f1': 0.49574608020795713,
               'time_taken': 325.0428841114044}}})

## Use bert classifier with classification head

This technique is pretty much same to the previous method. But here we finetune our model. Let's see how its done. Huggingface offers a class named AutoModelForSequenceClassification which can be instantiated in the same way as we initialize other transformers. Then, what is the difference in this case. Earlier, we took the embeddings corresponding to the CLS token and built our own model. In this case Huggingface does it for us. A classification head is attached on the top of the CLS token that takes in the embedding vector and outputs the logits corresponding to the number of output classes (in this case 6 classes). You might think decoder only models like LLAMA don't have a CLS token. In that case, the last token which was able to attend to all the previous tokens in the sentence (decoder only models don't have bi-directional attention) will act as input to our classifier. We take this model and finetune it for our use case. Finetuning is also simplified by the use of Trainer library by Huggingface.

I am not explaining the parameters very much. There are a lot of blogs that explain them in an detailed manner.

In [ ]:
model_name = 'microsoft/deberta-v3-base'
config.checkpoint = model_name
config.model_name = "deberta"

model, tokenizer = load_model_and_tokenizer(config=config,
                                            add_pad_token=False,
                                            peft=False,
                                            load_model_for_sequence_classification=True
                                            )

ds = tokenize_dataset(tokenizer,
                      config)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

...

Map:   0%|          | 0/1056 [00:00<?, ? examples/s]

Map:   0%|          | 0/3145 [00:00<?, ? examples/s]

Map:   0%|          | 0/16771 [00:00<?, ? examples/s]

Map:   0%|          | 0/1056 [00:00<?, ? examples/s]

Map:   0%|          | 0/3145 [00:00<?, ? examples/s]

In [ ]:
f1_score_results, time_taken = generate_predictions_using_bert_models(model,ds['test'])
complete_results['bert_classifier_without_finetuning'][model_name] = {**f1_score_results, 'time_taken': time_taken}

## Finetuning

This method

In [ ]:
model_name = 'microsoft/deberta-v3-base'
model_name = model_name.split('/')[-1]
config = update_model_related_settings(model_name, config)

model, tokenizer = load_model_and_tokenizer(config=config,
                                            quantization=False,
                                            add_pad_token=False,
                                            peft=True,
                                            load_model_for_sequence_classification=True
                                            )

ds = tokenize_dataset(tokenizer,
                      config)

finetuned_model = finetune_model(model,tokenizer,ds,config)

upload_model_to_huggingface(finetuned_model=finetuned_model, tokenizer=tokenizer, config=config, peft=True, quantized=False)

No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.


spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

In [ ]:
model_name = "bhujith10/deberta_finetuned_model_2024_11_23_12_22"
config = update_model_related_settings(model_name, config)

model, tokenizer = load_model_and_tokenizer(config=config,
                                            quantization=False,
                                            add_pad_token=False,
                                            peft=False,
                                            load_model_for_sequence_classification=True
                                            )

f1_score_results, time_taken = generate_predictions_using_bert_models(finetuned_model,ds['test'])
complete_results['bert_classifier_with_finetuning'][config.model_name] = {**f1_score_results, 'time_taken': time_taken}

tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [ ]:
(sigmoid(outputs.logits.detach().type(torch.float).cpu().numpy())>0.45).astype(float)

array([[1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.]])

In [ ]:
# To use while encoding
# Use stored embeddings

# from sentence_transformers import SentenceTransformer

# model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
# model.to(device)

# def get_embeddings(batch):
#   embeddings = model.encode(batch['text'], convert_to_tensor=True)
#   return {"embeddings": embeddings}

# ds.set_format("torch")
# ds_encoded = ds.map(get_embeddings, batched=True, batch_size=4)

Map:   0%|          | 0/16771 [00:00<?, ? examples/s]

Map:   0%|          | 0/4201 [00:00<?, ? examples/s]

In [ ]:
from datasets import Dataset
import re
import ast

def convert_dataframe_into_dataset(filepath):
  df = pd.read_csv(filepath)
  df['labels'] = df['labels'].apply(lambda x: ast.literal_eval(re.sub(r'\s+',' ',x).replace("[ ","[").replace(" ",",")))
  df['embeddings'] = df['embeddings'].apply(lambda x: ast.literal_eval(re.sub(r'\s+',' ',x).replace("[ ","[").replace(" ",",")))
  df = Dataset.from_pandas(df)
  return df

In [ ]:
ds_train = convert_dataframe_into_dataset('/content/drive/MyDrive/multilabel_classification_dataset/train_all_minilm_v6_embeddings.csv')
ds_val = convert_dataframe_into_dataset('/content/drive/MyDrive/multilabel_classification_dataset/validation_all_minilm_v6_embeddings.csv')

In [ ]:
from datasets import DatasetDict
ds = DatasetDict(
    {
        "train": ds_train,
        "val": ds_val
    }
)

In [ ]:
ds.set_format("torch")
ds_encoded = ds.map(lambda x: {"labels_f": x["labels"].to(torch.float)},remove_columns=["text","labels"])
ds_encoded = ds_encoded.rename_column("labels_f", "labels")

Map:   0%|          | 0/16771 [00:00<?, ? examples/s]

Map:   0%|          | 0/4201 [00:00<?, ? examples/s]

In [ ]:
train_actual_labels = ds['train']['labels'][:].numpy()
val_actual_labels = ds['val']['labels'][:].numpy()

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [ ]:
from scipy.special import expit as sigmoid
from sklearn.metrics import classification_report

label_constructions = [
    lambda label_name: f'{label_name}',
    lambda label_name: f'This article is about "{label_name}".',
    lambda label_name: f'This text discusses the topic of "{label_name}" in detail.'
]

results = []

for strategy in label_constructions:

  label_texts = [strategy(label_name) for label_name in label_names]

  label_embeddings = model.encode(label_texts, convert_to_tensor=True)

  predicted_labels = []

  for text_embedding in ds_encoded['train']['embeddings']:
      text_embedding = text_embedding.unsqueeze(0)
      similarities = F.cosine_similarity(text_embedding, label_embeddings, dim=1)
      y_pred = sigmoid(similarities).numpy()
      predicted_label = (y_pred > 0.5).astype(float)
      predicted_labels.append(predicted_label)

  report = classification_report(train_actual_labels, predicted_labels, output_dict=True, target_names=label_names)

  f1_score = report['weighted avg']['f1-score']
  precision = report['weighted avg']['precision']
  recall = report['weighted avg']['recall']
  results.append({
      'Rephrased Label': strategy('LABEL'),
      'F1 Score': f1_score,
      'Precision': precision,
      'Recall': recall,
      'Complete_Report':report
  })


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
results[2]

{'Rephrased Label': 'This text discusses the topic of "LABEL" in detail.',
 'F1 Score': 0.5357256757046591,
 'Precision': 0.3988499429798412,
 'Recall': 0.8734592871079808,
 'Complete_Report': {'Computer Science': {'precision': 0.5387803979366249,
   'recall': 0.8507636363636364,
   'f1-score': 0.6597484631436468,
   'support': 6875.0},
  'Physics': {'precision': 0.3558903643173306,
   'recall': 0.8692307692307693,
   'f1-score': 0.5050126826911463,
   'support': 4810.0},
  'Mathematics': {'precision': 0.3366453749704282,
   'recall': 0.9499332443257676,
   'f1-score': 0.497117903930131,
   'support': 4494.0},
  'Statistics': {'precision': 0.342609397055858,
   'recall': 0.8158463385354142,
   'f1-score': 0.48256763473691683,
   'support': 4165.0},
  'Quantitative Biology': {'precision': 0.04638760313946468,
   'recall': 0.9808510638297873,
   'f1-score': 0.08858570330514988,
   'support': 470.0},
  'Quantitative Finance': {'precision': 0.017242896102753584,
   'recall': 0.984924623115

In [ ]:
classification_report(train_actual_labels[:2],predicted_labels)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


'              precision    recall  f1-score   support\n\n           0       0.50      1.00      0.67         1\n           1       0.00      0.00      0.00         0\n           2       0.50      1.00      0.67         1\n           3       0.00      0.00      0.00         0\n           4       0.00      0.00      0.00         0\n           5       0.00      0.00      0.00         0\n\n   micro avg       0.22      1.00      0.36         2\n   macro avg       0.17      0.33      0.22         2\nweighted avg       0.50      1.00      0.67         2\n samples avg       0.23      1.00      0.37         2\n'

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report

# Define label construction strategies
label_constructions = [
    lambda label_name: f'{label_name}',
    lambda label_name: f'This article is about "{label_name}".',
    lambda label_name: f'This text discusses the topic of "{label_name}" in detail.'
]

results = []
# Iterate over label construction strategies
for strategy in label_constructions:
    # Construct label texts using the strategy
    label_texts = [strategy(label_name) for label_name in label_names]

    # Get embeddings for labels
    label_embeddings = get_embeddings([{"text": text} for text in label_texts], model=model_name)

    # Classify texts based on maximum cosine similarity with label embeddings
    predicted_labels = []
    for text_embedding in text_embeddings:
        similarities = [cosine_similarity(text_embedding, label_embedding) for label_embedding in label_embeddings]
        predicted_label = np.argmax(similarities)
        predicted_labels.append(predicted_label)

    # Calculate classification metrics
    true_labels = labels
    report = classification_report(true_labels, predicted_labels, output_dict=True, target_names=label_names)

    f1_score = report['weighted avg']['f1-score']
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    results.append({
        'Rephrased Label': strategy('LABEL'),
        'F1 Score': f1_score,
        'Precision': precision,
        'Recall': recall,
    })




****************

In last article, we explored three different techniques for multi-label classification. In this article, we will explore a few more techniques.

Last time, we saw finetuning transformers with classification head. The creation of ChatGPT by OpenAI, Gemini by Google and Llama by Meta has created a huge shift in the field of AI. The basis of all these models is the transformer architecture and this revolutionary architecture was published first by the research team at Google in 2017. But, why there is so much noise now? When transformer was launched in 2017, it was published as a research paper and it did not get into the hands of consumers. But today, there would not be a single employee working in your company who might not have played with ChatGPT. From code generation, to image generation, to content creation, and what not. These models have started taking giant strides. Even there is fear of job loss. They have entered data scientist life to such an extent that, even for a simple classification task we have started using the OpenAI API's that help us to access the GPT models.

These LLM's have been trained on almost the entire web and hence they have very good generalized knowledge. But they might not be very good at specialized tasks. Thanks to the researchers and open source community who have been able to figure out ways to adapt these LLM's to our specific use cases. Especially huge shoutout to Meta, Mistral and a few more organizations for open sourcing their models compared to a few other organizations who have Open in their name and yet behave the opposite. Coming back to the topic. In our usecase, given the title and abstract of an article, we can use OpenAI API or Anthropic API or other such models to make the predictions. But let's take a step further. Let's finetune the LLM to our specific use case. There are a few different ways in which finetuning can be performed. Here, I will perform supervised finetuning. This aspect

### Function to convert labels into probabilities

In [ ]:
def convert_labels_into_probabilities(ds, split_name='train'):
  ds_pandas = ds[split_name][:]
  ds_pandas['modified_labels'] = ds_pandas['labels'].apply(lambda x:[random.uniform(0,0.2) if i==0 else random.uniform(0.9,1) for i in x])
  ds_torch = Dataset.from_pandas(ds_pandas)
  ds_torch.set_format("torch")
  return ds_torch

### Function to format text as instruction

In [ ]:
def format_instruction_llama(sample):
  return {"text":f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>:

  Given the title and abstract of a research paper, classify it into one or more of the following subjects based on its content: ['Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance']. Each subject label should be assigned a probability from 0 to 1 that represents the likelihood of the paper belonging to that subject.
  The output should be formatted as JSON, with each label as a key and its probability as the value.
  For example, if a paper is very likely to be classified as 'Physics' but less likely for 'Mathematics,' provide higher probability for 'Physics' and a lower probability for 'Mathematics.' Ensure that all 6 labels are included in the JSON, even if some probabilities are low. <|eot_id|>

  <|start_header_id|>user<|end_header_id|>:

  {sample['text']} <|eot_id|>

  <|start_header_id|>assistant<|end_header_id|>

  Response:
  {{
  "Computer Science": {sample['modified_labels'][0]},
  "Physics": {sample['modified_labels'][1]},
  "Mathematics": {sample['modified_labels'][2]},
  "Statistics": {sample['modified_labels'][3]},
  "Quantitative Biology": {sample['modified_labels'][4]},
  "Quantitative Finance": {sample['modified_labels'][5]}
}} <|end_of_text|>
  """
  }

In [ ]:
def format_instruction(sample):
  return {"text":f"""### Instruction:
  Given the title and abstract of a research paper, classify it into one or more of the following subjects based on its content: ['Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance']. Each subject label should be assigned a probability from 0 to 1 that represents the likelihood of the paper belonging to that subject.
  The output should be formatted as JSON, with each label as a key and its probability as the value.
  For example, if a paper is very likely to be classified as 'Physics' but less likely for 'Mathematics,' provide higher probability for 'Physics' and a lower probability for 'Mathematics.' Ensure that all 6 labels are included in the JSON, even if some probabilities are low.

  ### Input:
  {sample['text']}

  ### Response:
  {{
  "Computer Science": {sample['modified_labels'][0]},
  "Physics": {sample['modified_labels'][1]},
  "Mathematics": {sample['modified_labels'][2]},
  "Statistics": {sample['modified_labels'][3]},
  "Quantitative Biology": {sample['modified_labels'][4]},
  "Quantitative Finance": {sample['modified_labels'][5]}
}}</s>
  """
  }

### Function to format dataset

In [ ]:
def format_dataset(dataset):
  dataset_formatted = dataset.map(format_instruction_llama)
  columns_to_be_removed = [i for i in dataset.features.keys() if i!='text']
  dataset_formatted = dataset_formatted.remove_columns(columns_to_be_removed)
  # dataset_formatted = dataset_formatted.map(tokenize_function, batched=True, batch_size=None)

  return dataset_formatted

### Function to tokenize dataset

In [ ]:
def tokenize_function(sample):
  return tokenizer(sample["text"], padding=False, truncation=True, max_length=config.max_length, return_tensors="pt")

### Function to finetune model

In [ ]:
def finetune_model(model,tokenizer,dataset,config):
  num_train_epochs = config.num_train_epochs
  train_batch_size = config.batch_size
  gradient_accumulation_steps = config.gradient_accumulation_steps
  max_steps = int((len(ds['train'])*num_train_epochs)/(train_batch_size*gradient_accumulation_steps))

  training_arguments = TrainingArguments(
      per_device_train_batch_size=8,
      gradient_accumulation_steps=2,
      gradient_checkpointing=True,
      learning_rate=1e-4,
      fp16=config.fp16,
      bf16=config.bf16,
      logging_steps=1,
      output_dir="outputs",
      optim="paged_adamw_32bit",
      max_grad_norm=0.3,
      num_train_epochs=1,
      evaluation_strategy="epoch",
      eval_steps=1,
      save_steps=1,
      save_strategy="epoch",
      warmup_ratio=0.05,
      report_to="wandb",
      run_name=f"{config.repo_user_id}/{config.model_name}_results",
      lr_scheduler_type="cosine"
      )

  # Set supervised fine-tuning parameters
  trainer = SFTTrainer(
      model=model,
      dataset_batch_size=8,
      train_dataset=ds["train"],
      eval_dataset=ds["val"],
      dataset_text_field="text",
      max_seq_length=config.max_length,
      tokenizer=tokenizer,
      args=training_arguments
  )

  trainer.model.save_pretrained(config.local_save_path)

  return trainer.model


### Function to get quantization config

In [ ]:
def quantization_config(config):
  bnb_config = BitsAndBytesConfig(
      load_in_4bit=config.load_in_4bit,
      bnb_4bit_quant_type=config.bnb_4bit_quant_type,
      bnb_4bit_compute_dtype=config.bnb_4bit_compute_dtype,
      bnb_4bit_use_double_quant=config.bnb_4bit_use_double_quant,
  )
  return bnb_config

### Function to load model and tokenizer

In [ ]:
def load_model_and_tokenizer(config,
                             add_pad_token=True,
                             quantization=False,
                             peft=False):

  tokenizer = AutoTokenizer.from_pretrained(config.checkpoint, add_prefix_space=True)
  if add_pad_token:
    tokenizer.add_special_tokens({"pad_token":"<pad>"})

  model = AutoModelForCausalLM.from_pretrained(
    config.checkpoint,
    use_safetensors=True,
    quantization_config=quantization_config(config) if quantization else None,
    torch_dtype=torch.bfloat16 if config.bf16 else torch.float16,
    device_map="auto",
    use_cache=False,
    pretraining_tp=1,
    trust_remote_code=True
  )
  if add_pad_token:
    model.resize_token_embeddings(len(tokenizer))
    model.config.pad_token_id = tokenizer.pad_token_id

  if peft:
    # Load LoRA configuration
    peft_config = LoraConfig(
        r=config.lora_rank,
        lora_alpha=config.lora_alpha,
        lora_dropout=config.lora_dropout,
        bias=config.bias,
        task_type="CAUSAL_LM",
        target_modules=['q_proj', 'v_proj']
    )
    model = get_peft_model(model,peft_config=peft_config)

    return model, tokenizer

### Function to update model related config

In [ ]:
def update_model_related_settings(model_name, config):
  config.checkpoint = model_name
  currtime = datetime.now().strftime("%Y_%m_%d_%H_%M")
  config.model_name = f"{model_name}_{currtime}"
  config.local_save_path = config.model_name
  return config

### Function to upload model to huggingface hub

In [ ]:
def upload_model_to_huggingface(finetuned_model, config, tokenizer=None, peft=False, quantized=False):
  if peft and not quantized:
    finetuned_model.push_to_hub(f"{config.repo_user_id}/{config.model_name}_adapter", safe_serialization=True, max_shard_size='1GB')
    model = finetuned_model.merge_and_unload()

  elif peft and quantized:
    base_model, tokenizer = load_model_and_tokenizer(config,
                                                     quantization=False,
                                                     add_pad_token=False,
                                                     peft=False,
                                                     load_model_for_sequence_classification=True)
    model = PeftModel.from_pretrained(base_model, finetuned_model)
    model = model.merge_and_unload()

  else:
    model = finetuned_model

  model.push_to_hub(f"{config.repo_user_id}/{config.model_name}", safe_serialization=True, max_shard_size='1GB')
  tokenizer.push_to_hub(f"{config.repo_user_id}/{config.model_name}")

## Finetune generative transformer models

In [ ]:
@dataclass
class Config:
  checkpoint:str = "microsoft/deberta-v3-base"
  max_length:int = 512
  num_labels:int = 6
  problem_type:str = "multi_label_classification"
  lora_rank:int = 8
  lora_alpha:int = 32
  lora_dropout:float = 0.1
  lora_bias:str = "none"
  device:str = "cuda" if torch.cuda.is_available() else "cpu"
  repo_user_id:str = "bhujith10"
  model_name:str = ""
  local_save_path:str = ""
  bf16:bool = torch.cuda.is_available() and torch.cuda.is_bf16_supported()
  fp16:bool = torch.cuda.is_available() and not torch.cuda.is_bf16_supported()
  load_in_4bit:bool = True,
  bnb_4bit_quant_type:bool = "nf4",
  bnb_4bit_compute_dtype:bool = "float16",
  bnb_4bit_use_double_quant:bool = False,
  num_train_epochs:int = 2,
  batch_size:int = 8,
  gradient_accumulation_steps:int = 2,
  gradient_checkpointing:bool = True

config = Config()

In [ ]:
ds = load_dataset('bhujith10/multi_class_classification_dataset')

In [ ]:
ds.set_format("pandas")
ds_processed = DatasetDict()

for splitname in ds.keys():
  ds_processed[splitname] = convert_labels_into_probabilities(ds, splitname)

ds_processed.set_format('torch')

In [ ]:
train_dataset = format_dataset(ds_processed['train'])
eval_dataset = format_dataset(ds_processed['val'])

In [ ]:
checkpoint = "HuggingFaceTB/SmolLM-135M-Instruct"
config.checkpoint=checkpoint
model, tokenizer = load_model_and_tokenizer(config,
                                            add_pad_token=True,
                                            quantize=False,
                                            peft=True)

config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [ ]:
currtime = datetime.now().strftime("%Y_%m_%d_%H_%M")
config.model_name = f"deberta_finetuned_model_{currtime}"

finetuned_model = finetune_model(model,tokenizer,train_dataset,eval_dataset,config,push_to_hub=True)

In [ ]:
f1_score_results, time_taken = generate_predictions_using_bert_models(finetuned_model,ds['test'])
complete_results['bert_classifier_with_finetuning'][config.model_name] = {**f1_score_results, 'time_taken': time_taken}

In [ ]:
prompt = """
### Instruction:
  Given the title and abstract of a research paper, classify it into one or more of the following subjects based on its content: ['Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance']. Each subject label should be assigned a probability from 0 to 1 that represents the likelihood of the paper belonging to that subject.
  The output should be formatted as JSON, with each label as a key and its probability as the value.
  For example, if a paper is very likely to be classified as 'Physics' but less likely for 'Mathematics,' provide higher probability for 'Physics' and a lower probability for 'Mathematics.' Ensure that all 6 labels are included in the JSON, even if some probabilities are low.

### Input:
  Title: Complete Cyclic Proof Systems for Inductive Entailments,
  Abstract: In this paper we develop cyclic proof systems for the problem of inclusion
  between the least sets of models of mutually recursive predicates, when the
  ground constraints in the inductive definitions belong to the quantifier-free
  fragments of (i) First Order Logic with the canonical Herbrand interpretation
  and (ii) Separation Logic, respectively. Inspired by classical
  automata-theoretic techniques of proving language inclusion between tree
  automata, we give a small set of inference rules, that are proved to be sound
  and complete, under certain semantic restrictions, involving the set of
  constraints in the inductive system. Moreover, we investigate the decidability
  and computational complexity of these restrictions for all the logical
  fragments considered and provide a proof search semi-algorithm that becomes a
  decision procedure for the entailment problem, for those systems that fulfill
  the restrictions.
"""

In [ ]:
from transformers import pipeline

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_new_tokens=200)
result = pipe(f"{prompt}")
print(result[0]['generated_text'])


### Instruction:
  Given the title and abstract of a research paper, classify it into one or more of the following subjects based on its content: ['Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance']. Each subject label should be assigned a probability from 0 to 1 that represents the likelihood of the paper belonging to that subject.
  The output should be formatted as JSON, with each label as a key and its probability as the value.
  For example, if a paper is very likely to be classified as 'Physics' but less likely for 'Mathematics,' provide higher probability for 'Physics' and a lower probability for 'Mathematics.' Ensure that all 6 labels are included in the JSON, even if some probabilities are low.

### Input:
  Title: Complete Cyclic Proof Systems for Inductive Entailments,
  Abstract: In this paper we develop cyclic proof systems for the problem of inclusion
  between the least sets of models of mutually recursive predicat

In [ ]:
trainer

In [ ]:
from transformers import pipeline

pipe = pipeline(task="text-generation", model=trainer.model, tokenizer=tokenizer, max_new_tokens=200)
result = pipe(f"{prompt}")
print(result[0]['generated_text'])


### Instruction:
  Given the title and abstract of a research paper, classify it into one or more of the following subjects based on its content: ['Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance']. Each subject label should be assigned a probability from 0 to 1 that represents the likelihood of the paper belonging to that subject.
  The output should be formatted as JSON, with each label as a key and its probability as the value.
  For example, if a paper is very likely to be classified as 'Physics' but less likely for 'Mathematics,' provide higher probability for 'Physics' and a lower probability for 'Mathematics.' Ensure that all 6 labels are included in the JSON, even if some probabilities are low.

### Input:
  Title: Complete Cyclic Proof Systems for Inductive Entailments,
  Abstract: In this paper we develop cyclic proof systems for the problem of inclusion
  between the least sets of models of mutually recursive predicat

In [ ]:
from transformers import AutoTokenizer

class ModelForSequenceClassification:
  def __init__(self,checkpoint,add_pad_token=False,quantize=False):
    self.checkpoint = checkpoint
    self.add_pad_token = add_pad_token
    self.max_len = 512

    self.quantize = quantize
    if self.quantize:
      self.quant_config = BitsAndBytesConfig(
          load_in_4bit=True,
          bnb_4bit_quant_type="nf4",
          bnb_4bit_compute_dtype=torch.float16,
          bnb_4bit_use_double_quant=True
          )
    else:
      self.quant_config = None

  def __call__(self):
    self.create_tokenizer()
    self.create_model()
    return self.model, self.tokenizer

  def create_tokenizer(self):
    self.tokenizer = AutoTokenizer.from_pretrained(self.checkpoint, add_prefix_space=True)
    if self.add_pad_token:
        self.tokenizer.add_special_tokens({"pad_token":"<pad>"})
        self.tokenizer.pad_token = "<pad>"

  def tokenize_function(self, samples, truncation=True):
    return self.tokenizer(samples['text'], truncation=truncation, max_length=self.max_len)

  def tokenize(self, dataset):
    tokenized_dataset = dataset.map(self.tokenize_function, batched=True)
    tokenized_dataset.set_format("torch")
    tokenized_dataset = tokenized_dataset.map(lambda x: {"labels_f": x["labels"].to(torch.float)},remove_columns=["text","labels"])
    tokenized_dataset = tokenized_dataset.rename_column("labels_f", "labels")
    return tokenized_dataset

  def create_model(self):
    self.model = AutoModelForCausalLM.from_pretrained(
        self.checkpoint,
        use_safetensors=True,
        quantization_config=self.quant_config,
        device_map="auto",
        trust_remote_code=True
    )
    if self.add_pad_token:
      self.model.resize_token_embeddings(len(self.tokenizer))
      self.model.config.pad_token_id = self.tokenizer.pad_token_id

In [ ]:
roberta_checkpoint = "roberta-large"
tokenizer = Tokenizer(roberta_checkpoint, add_pad_token=False)
ds_tokenized = tokenizer.tokenize(ds)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/16771 [00:00<?, ? examples/s]

Map:   0%|          | 0/4201 [00:00<?, ? examples/s]

Map:   0%|          | 0/16771 [00:00<?, ? examples/s]

Map:   0%|          | 0/4201 [00:00<?, ? examples/s]

In [ ]:
tokenizer.tokenizer.pad_token_id

1